In [ ]:
import pandas as pd
import requests
import time
import numpy as np
import bs4

In [18]:
def make_fs_dataframe(firm_code): #재무데이터 만드는 함수
    fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode=' + firm_code 
    fs_page = requests.get(fs_url) 
    fs_tables = pd.read_html(fs_page.text) 
 
    temp_df = fs_tables[0] 
    temp_df = temp_df.set_index(temp_df.columns[0]) 
    temp_df = temp_df[temp_df.columns[:4]] 
    temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']] 

    temp_df2 = fs_tables[2] 
    temp_df2 = temp_df2.set_index(temp_df2.columns[0]) 
    temp_df2 = temp_df2.loc[['자산', '자본', '부채']] 

    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

    fs_df = pd.concat([temp_df, temp_df2, temp_df3])
 
    return fs_df 

In [19]:
def make_fr_dataframe(firm_code): #재무비율 데이터 만들기함수
    fr_url = 'https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode=' + firm_code 
    fr_page = requests.get(fr_url) 
    fr_tables = pd.read_html(fr_page.text) 
    
    temp_df = fr_tables[0] 
    temp_df = temp_df.set_index(temp_df.columns[0]) 
    temp_df = temp_df.loc[['유동비율계산에 참여한 계정 펼치기',
                           '부채비율계산에 참여한 계정 펼치기',
                           '영업이익률계산에 참여한 계정 펼치기',
                           'ROA계산에 참여한 계정 펼치기',
                           'ROIC계산에 참여한 계정 펼치기']] 
    temp_df.index = ['유동비율', '부채비율', '영업이익률', 'ROA', 'ROIC']  
    return temp_df

In [20]:
def make_invest_dataframe(firm_code): #투자지표 만드는 함수
    invest_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode=' + firm_code 
    invest_page = requests.get(invest_url) 
    invest_tables = pd.read_html(invest_page.text) 
    temp_df = invest_tables[1] 
    
    temp_df = temp_df.set_index(temp_df.columns[0]) 
    temp_df = temp_df.loc[['PER계산에 참여한 계정 펼치기',
                           'PCR계산에 참여한 계정 펼치기',
                           'PSR계산에 참여한 계정 펼치기',
                           'PBR계산에 참여한 계정 펼치기',
                          '총현금흐름']]  
    temp_df.index = ['PER', 'PCR', 'PSR', 'PBR', '총현금흐름'] 
    return temp_df

In [21]:
def change_df(firm_code, dataframe): #데이터프레임의 행과열을 바꾸는 함수 만듬 
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code : dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]
        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)
    
    return total_df   

In [29]:
def come_code(): #종목코드 불러오기, 종목코드 전처리
    path = r'C:\Users\gjals\Desktop\코딩\data.xls'
    code_data = pd.read_excel(path)  
    code_data = code_data[['종목코드', '기업명']] 

    def make_code(x):  
        x = str(x)  
        return 'A' + '0' * (6-len(x)) + x

    code_data['종목코드'] = code_data['종목코드'].apply(make_code)
        
    return code_data

In [23]:
def make_total_fs_df(): #전종목 재무데이터 크롤링, 저장
    for num, code in enumerate(code_data['종목코드']): 
        try: 
            print(num,code) 
            time.sleep(1) 
            try: 
                fs_df = make_fs_dataframe(code)  
            except requests.exceptions.Timeout: 
                time.sleep(60) 
                fs_df = make_fs_dataframe(code) 
            fs_df_changed = change_df(code, fs_df) 
            if num == 0: 
                total_fs =fs_df_changed 
            else:
                total_fs = pd.concat([total_fs, fs_df_changed]) 
        except ValueError: 
            continue 
        except KeyError: 
            continue 

    total_fs.to_excel(r'C:\Users\gjals\Desktop\코딩\재무데이터test.xlsx')   

In [24]:
def make_total_fr_df(): #전종목 재무비율데이터 크롤링, 저장
    for num, code in enumerate(code_data['종목코드']): 
        try:
            print(num, code)
            time.sleep(1)
            try:
                fr_df = make_fr_dataframe(code)
            except requests.exceptions.Timeout:
                time.sleep(60)
                fr_df = make_fr_dataframe(code)
            fr_df_changed = change_df(code, fr_df)
            if num == 0:
                total_fr = fr_df_changed
            else:
                total_fr = pd.concat([total_fr, fr_df_changed])
        except ValueError:
            continue
        except KeyError:
            continue
        
    total_fr.to_excel(r'C:\Users\gjals\Desktop\코딩\재무비율데이터test.xlsx')

In [25]:
def make_total_invest_df(): #전종목 투자지표 크롤링, 저장
    for num,code in enumerate(code_data['종목코드']):
        try:
            print(num, code)
            time.sleep(1)
            try:
                invest_df = make_invest_dataframe(code)
            except requests.exceptions.Timeout:
                time.sleep(60)
                invest_df = make_invest_dataframe(code)
            invest_df_changed = change_df(code, invest_df)
            if num == 0:
                total_invest = invest_df_changed
            else:
                total_invest = pd.concat([total_invest, invest_df_changed])
        except ValueError:
            continue
        except KeyError:
            continue

    total_invest.to_excel(r'C:\Users\gjals\Desktop\코딩\투자지표데이터test.xlsx')

In [26]:
def make_price_dataframe(code, timeframe, count): #가격데이터 함수지정 종목코드, 시간, 갯수
    url = 'https://fchart.stock.naver.com/sise.nhn?requestType=0'
    price_url = url + '&symbol=' + code + '&timeframe=' + timeframe + '&count=' + count 

    price_data = requests.get(price_url) 
    price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml') 
    item_list = price_data_bs.find_all('item') 
    date_list =[] 
    price_list = [] 
    for item in item_list: 
        temp_data = item['data']
        datas = temp_data.split('|') 
        date_list.append(datas[0]) 
        price_list.append(datas[4]) 
      
    price_df = pd.DataFrame({code : price_list}, index=date_list) 

    return price_df

In [27]:
def make_total_price_df(): #전종목 가격데이터 크롤링, 저장
    for num, code in enumerate(code_data['종목코드']): 
        try: 
            print(num, code) 
            time.sleep(1) 
            try: 
                price_df = make_price_dataframe(code, 'day', '1500') 
            except requests.exception.Timeout: 
                time.sleep(60) 
                price_df = make_price_dataframe(code, 'day', '1500') 
            if num == 0: 
                total_price = price_df 
            else:
                total_price = pd.merge(total_price, price_df, how='outer', right_index=True, left_index=True) 
        except ValueError: 
            continue 
        except KeyError: 
            continue 
    
    total_price.index = pd.to_datetime(total_price.index) #날짜를 날짜형 인덱스로 바꿔줌 ex)20201029 -> 2020-10-29
    total_price.to_excel(r'C:\Users\gjals\Desktop\코딩\가격데이터test.xlsx') 

In [ ]:
#데이터 전처리 개선
def get_finance_data(path):
    data_path = path
    raw_data = pd.read_excel(data_path)
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])
    
    big_col[0] = '종목'
    small_col[0] = ''
    new_big_col = []
    for num, col in enumerate(big_col):
        if 'Unnamed' in col:
            new_big_col.append(new_big_col[num-1])
        else:
            new_big_col.append(big_col[num])

    raw_data.columns = [new_big_col, small_col]
    raw_data = raw_data.drop(0)
    clean_df = raw_data.drop(1)

    if clean_df.index[0]  == clean_df['종목'][2]:
         pass
    else:
        for i in clean_df.index:
            clean_df = clean_df.rename(index={i : clean_df['종목'][i]})

    clean_df = clean_df.drop(clean_df.columns[0:1], axis=1, inplace=False)

    return clean_df

In [ ]:
fs_path = r'C:\Users\gjals\Desktop\코딩\재무데이터.xlsx'
fs_df = get_finance_data(fs_path)

In [ ]:
fr_path = r'C:\Users\gjals\Desktop\코딩\재무비율데이터.xlsx'
fr_df = get_finance_data(fr_path)

In [ ]:
invest_path = r'C:\Users\gjals\Desktop\코딩\투자지표데이터.xlsx'
invest_df = get_finance_data(invest_path)

In [ ]:
def check_IFRS(x): #N/A_IFRS(x)를 Nan으로 바꾸기
    if x == 'N/A(IFRS)':
        return np.NaN
    else:
        return x

In [ ]:
def low_per(invest_df, index_date, num): #per기준으로 오름차순 정렬
    invest_df[(index_date, 'PER')] = pd.to_numeric(invest_df[(index_date, 'PER')])
    per_sorted = invest_df.sort_values(by=(index_date, 'PER'))
    return per_sorted[index_date][:num]

In [ ]:
def high_roa(fr_df, index_date, num): #roa기준으로 내림차순 정렬
    fr_df[(index_date, 'ROA')] = fr_df[(index_date, 'ROA')].apply(check_IFRS)
    fr_df[(index_date, 'ROA')] = pd.to_numeric(fr_df[(index_date, 'ROA')])
    sorted_roa = fr_df.sort_values(by=(index_date, 'ROA'), ascending=False)
    return sorted_roa[index_date][:num]

In [ ]:
def magic_formula(fr_df, invest_df, index_date, num): #마법공식 저per, 고roa 합성 함수
    per = low_per(invest_df, index_date, None)
    roa = high_roa(fr_df, index_date, None)
    per['per순위'] = per['PER'].rank()
    roa['roa순위'] = roa['ROA'].rank(ascending=False)
    magic = pd.merge(per, roa, how='outer', left_index=True, right_index=True)
    magic['마법공식 순위'] = (magic['per순위'] + magic['roa순위']).rank().sort_values()
    magic = magic.sort_values(by='마법공식 순위')
    return magic[:num]

In [ ]:
def get_value_rank(invest_df, value_type, index_date, num): #저평가 지수 기준으로 정렬해 순위 만들어 주는 함수 
    invest_df[(index_date, value_type)] = pd.to_numeric(invest_df[(index_date, value_type)])
    value_sorted = invest_df.sort_values(by=(index_date, value_type))[index_date]
    value_sorted[value_type + '순위'] = value_sorted[value_type].rank()
    return value_sorted[[value_type, value_type + '순위']][:num]

In [ ]:
def make_value_combo(value_list, invest_df, index_date, num): #저평가 지표 조합 함수

    for i, value in enumerate(value_list):
        temp_df = get_value_rank(invest_df, value, index_date, None)
        if i == 0:
            value_combo_df = temp_df
            rank_combo = temp_df[value + '순위']
        else:
            value_combo_df = pd.merge(value_combo_df, temp_df, how='outer', right_index=True, left_index=True)
            rank_combo = rank_combo + temp_df[value + '순위']

    value_combo_df['종합순위'] = rank_combo.rank()
    value_combo_df = value_combo_df.sort_values(by='종합순위')

    return value_combo_df[:num]

In [ ]:
def get_fscore(fs_df, index_date, num): #f스코어
    fscore_df = fs_df[index_date]
    fscore_df['당기순이익점수'] = fscore_df['당기순이익'] > 0
    fscore_df['영업활동점수'] = fscore_df['영업활동으로인한현금흐름'] > 0
    fscore_df['더큰영업활동점수'] = fscore_df['영업활동으로인한현금흐름'] > fscore_df['당기순이익']
    fscore_df['종합점수'] = fscore_df[['당기순이익점수', '영업활동점수', '더큰영업활동점수']].sum(axis=1)
    fscore_df = fscore_df[fscore_df['종합점수'] == 3]
    return fscore_df[:num]

In [13]:
price_path = r'C:\Users\gjals\Desktop\코딩\가격데이터.xlsx'  
price_df = pd.read_excel(price_path)

#가격데이터 전처리함수
for i in range(0, 1500):
    price_df = price_df.rename(index={i : price_df['Unnamed: 0'][i]})
    
price_df=price_df.drop(price_df.columns[0:1], axis=1, inplace=False)

In [ ]:
def get_momentum_rank(price_df, index_date, date_range, num): #모멘텀 함수
    momentum_df = pd.DataFrame(price_df.pct_change(date_range).loc[index_date])
    momentum_df.columns = ['모멘텀']
    momentum_df['모멘텀순위'] = momentum_df['모멘텀'].rank(ascending=False)
    momentum_df = momentum_df.sort_values(by='모멘텀순위')
    return momentum_df[:num]

In [ ]:
def get_value_quality(invest_df, fs_df, index_date, num): #저평가 지수 + f스코어
    value = make_value_combo(['PER', 'PBR', 'PSR', 'PCR'], invest_df, index_date, None)
    quality = get_fscore(fs_df, index_date, None)
    value_quality = pd.merge(value, quality, how='outer', right_index=True, left_index=True)
    value_quality_filtered = value_quality[value_quality['종합점수'] == 3]
    vq_df = value_quality_filtered.sort_values(by='종합순위')
    return vq_df[:num]